In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import lightgbm as lgb
import xgboost as xgb

# 데이터 불러오기
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# X와 Y로 분리
x = train.drop(['ID', '허위매물여부'], axis=1)
y = train['허위매물여부']

# 날짜 처리
x['게재일'] = pd.to_datetime(x['게재일'], errors='coerce')

# 새로운 특성 생성
x['게재년도'] = x['게재일'].dt.year
x['게재월'] = x['게재일'].dt.month
x['게재일_주말여부'] = (x['게재일'].dt.dayofweek >= 5).astype(int)

# 수치형 특성 변환
x['보증금_월세비율'] = x['보증금'] / (x['월세'] + 1)
x['전용면적_방수비율'] = x['전용면적'] / (x['방수'] + 1)
x['총층_해당층비율'] = x['해당층'] / (x['총층'] + 1)

# 중개사무소 평균 보증금
avg_deposit_by_agency = x.groupby('중개사무소')['보증금'].mean().to_dict()
x['중개사무소_평균보증금'] = x['중개사무소'].map(avg_deposit_by_agency)

# 방향 원-핫 인코딩
x = pd.get_dummies(x, columns=['방향'], drop_first=True)

# 주차 가능 여부에 따른 평균 관리비
avg_management_fee_by_parking = x.groupby('주차가능여부')['관리비'].mean().to_dict()
x['주차가능여부_평균관리비'] = x['주차가능여부'].map(avg_management_fee_by_parking)

# 욕실 수 이진 특성
x['욕실수_이진'] = (x['욕실수'] >= 2).astype(int)

# 날짜 열 제거
x = x.drop(columns=['게재일'])

# 범주형 및 수치형 열 식별
numeric_cols = x.select_dtypes(include=[np.number]).columns
categorical_cols = x.select_dtypes(include=[object]).columns

# 특성 인코딩
label_encoders = {col: LabelEncoder().fit(x[col]) for col in categorical_cols}
for col in categorical_cols:
    x[col] = label_encoders[col].transform(x[col])

# 결측치 처리
numeric_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')
x[numeric_cols] = numeric_imputer.fit_transform(x[numeric_cols])
x[categorical_cols] = categorical_imputer.fit_transform(x[categorical_cols])

# SMOTE로 불균형 해결
smote = SMOTE(random_state=42)
x_resampled, y_resampled = smote.fit_resample(x, y)

# 앙상블 모델 구성
model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model2 = GradientBoostingClassifier(n_estimators=100, random_state=42)
model3 = lgb.LGBMClassifier(random_state=42)
model4 = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

ensemble_model = VotingClassifier(estimators=[('rf', model1), ('gb', model2), ('lgb', model3), ('xgb', model4)], voting='soft')

# 하이퍼파라미터 튜닝 예제
param_grid = {
    'rf__n_estimators': [50, 100, 200],
    'gb__n_estimators': [50, 100],
    'lgb__num_leaves': [31, 50],
    'xgb__max_depth': [3, 5, 7]
}

# GridSearchCV 설정
grid_search = GridSearchCV(ensemble_model, param_grid, cv=StratifiedKFold(n_splits=5), scoring='accuracy', n_jobs=-1)
grid_search.fit(x_resampled, y_resampled)

# 최적의 하이퍼파라미터 출력
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

# 전체 데이터로 모델 훈련
best_ensemble_model = grid_search.best_estimator_
best_ensemble_model.fit(x_resampled, y_resampled)

# 테스트 데이터 인코딩 및 결측치 처리
test['게재일'] = pd.to_datetime(test['게재일'], errors='coerce')
test['게재년도'] = test['게재일'].dt.year
test['게재월'] = test['게재일'].dt.month
test['게재일_주말여부'] = (test['게재일'].dt.dayofweek >= 5).astype(int)

# 수치형 특성 변환
test['보증금_월세비율'] = test['보증금'] / (test['월세'] + 1)
test['전용면적_방수비율'] = test['전용면적'] / (test['방수'] + 1)
test['총층_해당층비율'] = test['해당층'] / (test['총층'] + 1)
test['중개사무소_평균보증금'] = test['중개사무소'].map(avg_deposit_by_agency)

# 방향 원-핫 인코딩
test = pd.get_dummies(test, columns=['방향'], drop_first=True)

# 주차 가능 여부에 따른 평균 관리비
test['주차가능여부_평균관리비'] = test['주차가능여부'].map(avg_management_fee_by_parking)

# 욕실 수 이진 특성
test['욕실수_이진'] = (test['욕실수'] >= 2).astype(int)

# 날짜 열 제거
test = test.drop(columns=['게재일'])

# 범주형 및 수치형 열 식별
test[numeric_cols] = numeric_imputer.transform(test[numeric_cols])
test[categorical_cols] = categorical_imputer.transform(test[categorical_cols])

# 범주형 인코딩 처리
for col in categorical_cols:
    test[col] = test[col].map(lambda s: '<unknown>' if s not in label_encoders[col].classes_ else s)
    label_encoders[col].classes_ = np.append(label_encoders[col].classes_, '<unknown>')
    test[col] = label_encoders[col].transform(test[col])

# 예측 수행
pred = best_ensemble_model.predict(test.drop(columns=['ID']))

# 결과 제출
submit = pd.read_csv("sample_submission.csv")
submit['허위매물여부'] = pred
submit.to_csv("submission.csv", index=False)